In [1]:
from load import *
from torch import Tensor, nn
import torch
from model_base import *
from modules import *
from anim import *
import util
import matplotlib.pyplot as plt
from scipy.optimize import minimize

ROOT = "./Datas/Week 8"

Q = 1.60217663e-19

In [2]:
sc = load_space_charge() * -Q
ep = load_elec_potential()
vg = load_vgs()
poi = NormalizedPoissonMSE('cpu')
poi(ep, sc)

tensor(5.3032e-10, dtype=torch.float64)

In [4]:
predictions = {}
for index in util.TRAINING_IDXS:
    x = vg[index]
    epy = ep[index]
    scy = sc[index]
    ep_linear = TrainedLinear(1, 2193, algorithm='linear').fit(x, epy.reshape(-1, 2193))
    sc_linear = TrainedLinear(1, 2193, algorithm='linear').fit(x, scy.reshape(-1, 2193))
    model = PoissonJITRegressor(ep_linear, sc_linear)
    ypred = model(vg)
    ypred_ep = ypred[:, :2193].reshape(-1, 129, 17)
    predictions[index.name] = ypred_ep.detach().numpy()
    print(f"Trained {index.name}")

Trained First 5
Trained First 20
Trained First 30
Trained First 40
Trained First 60
Trained First 75
Trained First 90
Trained 15 to 45
Trained 20 to 40
Trained 40 to 60
Trained 25 to 35
Trained 20 to 50
Trained 30 to 50
Trained 29 and 30 and 31


In [8]:
save_h5(predictions, "./predictions.h5")